In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Amapá - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_AP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Amapá - Produção de Cimento (t)', axis=1)
data

,Ano,Amapá - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Amapá - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Longevidade,Amapá - Consumo de Cimento (t)
0,2008,0.685967,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,8.281094,0.717218,5.483335e+08,2.546644e+09,...,0.679138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.791218,116.3660
1,2009,0.687230,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,8.302950,0.717750,5.302764e+08,2.679945e+09,...,0.678150,23.388804,1009.883647,1230.481450,25.905555,23.095009,85.784466,0.981601,0.793078,112.7290
2,2010,0.688267,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,8.270873,0.718079,5.046425e+08,2.891930e+09,...,0.677213,20.607778,1009.759833,1208.062836,25.916677,23.199352,86.045978,0.939026,0.794442,151.5420
3,2011,0.689067,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,8.258737,0.718205,4.854622e+08,3.091988e+09,...,0.676323,19.725663,1009.881525,1245.286694,25.844771,23.091132,85.930050,0.977264,0.795319,161.9700
4,2012,0.688823,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,14.118134,0.717668,4.867661e+08,3.189506e+09,...,0.675105,19.226938,1010.130132,1288.178519,25.905589,23.183123,86.045636,1.031823,0.795637,261.1200
5,2013,0.691309,1.184231e+09,4.786423e+07,24.041932,1.003187e+09,14.922159,0.718558,5.225848e+08,3.093872e+09,...,0.675280,17.566310,1010.375134,1304.077775,25.869470,23.293712,86.546239,1.045650,0.795607,250.5190
6,2014,0.689418,1.197826e+09,4.735435e+07,24.001434,1.003012e+09,15.577783,0.717271,5.676413e+08,2.779340e+09,...,0.673970,19.134834,1010.606422,1297.941803,25.831023,23.260638,86.517021,1.031085,0.795157,227.2430
7,2015,0.689536,1.206909e+09,4.689336e+07,23.951008,1.001922e+09,16.255907,0.716627,5.179392e+08,2.561731e+09,...,0.672885,25.746659,1010.610813,1282.207220,25.934519,23.278348,86.261290,1.002171,0.794254,198.0696
8,2016,0.693917,1.212819e+09,4.650979e+07,23.922375,1.001305e+09,16.439693,0.719375,4.524144e+08,2.516859e+09,...,0.668385,29.398032,1010.471545,1283.328334,26.010134,23.247459,86.142602,0.972635,0.803514,166.8504
9,2017,0.692546,1.214213e+09,4.632537e+07,23.910733,1.000929e+09,16.090418,0.718136,4.213689e+08,2.497710e+09,...,0.667148,28.039516,1010.496689,1264.603524,25.817332,23.285926,87.487872,0.956773,0.802470,135.0210


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Amapá - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Amapá - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Longevidade
0,0.685967,1.092396e+09,4.853082e+07,24.143787,9.946828e+08,8.281094,0.717218,5.483335e+08,2.546644e+09,1.874883e+10,...,11.760983,0.679138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.791218
1,0.687230,1.098669e+09,4.873392e+07,24.141551,9.947096e+08,8.302950,0.717750,5.302764e+08,2.679945e+09,2.055015e+10,...,9.472585,0.678150,23.388804,1009.883647,1230.481450,25.905555,23.095009,85.784466,0.981601,0.793078
2,0.688267,1.124010e+09,4.884708e+07,24.146600,9.985217e+08,8.270873,0.718079,5.046425e+08,2.891930e+09,2.342247e+10,...,9.332485,0.677213,20.607778,1009.759833,1208.062836,25.916677,23.199352,86.045978,0.939026,0.794442
3,0.689067,1.146637e+09,4.854680e+07,24.134292,1.000762e+09,8.258737,0.718205,4.854622e+08,3.091988e+09,2.604265e+10,...,10.985689,0.676323,19.725663,1009.881525,1245.286694,25.844771,23.091132,85.930050,0.977264,0.795319
4,0.688823,1.166721e+09,4.823983e+07,24.089660,1.002188e+09,14.118134,0.717668,4.867661e+08,3.189506e+09,2.708753e+10,...,8.159014,0.675105,19.226938,1010.130132,1288.178519,25.905589,23.183123,86.045636,1.031823,0.795637
5,0.691309,1.184231e+09,4.786423e+07,24.041932,1.003187e+09,14.922159,0.718558,5.225848e+08,3.093872e+09,2.523394e+10,...,7.913561,0.675280,17.566310,1010.375134,1304.077775,25.869470,23.293712,86.546239,1.045650,0.795607
6,0.689418,1.197826e+09,4.735435e+07,24.001434,1.003012e+09,15.577783,0.717271,5.676413e+08,2.779340e+09,1.954361e+10,...,10.350442,0.673970,19.134834,1010.606422,1297.941803,25.831023,23.260638,86.517021,1.031085,0.795157
7,0.689536,1.206909e+09,4.689336e+07,23.951008,1.001922e+09,16.255907,0.716627,5.179392e+08,2.561731e+09,1.327596e+10,...,12.454398,0.672885,25.746659,1010.610813,1282.207220,25.934519,23.278348,86.261290,1.002171,0.794254
8,0.693917,1.212819e+09,4.650979e+07,23.922375,1.001305e+09,16.439693,0.719375,4.524144e+08,2.516859e+09,1.011370e+10,...,13.111908,0.668385,29.398032,1010.471545,1283.328334,26.010134,23.247459,86.142602,0.972635,0.803514
9,0.692546,1.214213e+09,4.632537e+07,23.910733,1.000929e+09,16.090418,0.718136,4.213689e+08,2.497710e+09,8.456309e+09,...,9.479812,0.667148,28.039516,1010.496689,1264.603524,25.817332,23.285926,87.487872,0.956773,0.802470


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     112.7290
1     151.5420
2     161.9700
3     261.1200
4     250.5190
5     227.2430
6     198.0696
7     166.8504
8     135.0210
9     123.3470
10    120.1530
11    133.5230
12    157.6210
13    142.7390
14         NaN
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input = train_input.fillna(0.0)
train_input

,Amapá - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Amapá - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Longevidade
0,-1.598841,-1.819408,1.003478,1.202107,-1.961368,-1.393611,-0.283888,1.204752,-0.990234,0.166091,...,0.984445,1.322153,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.489343
1,-1.043588,-1.675567,1.209427,1.179772,-1.951541,-1.387360,0.189618,0.820657,-0.546688,0.428911,...,-0.051710,1.112223,-0.024438,-1.344538,-0.680823,0.507454,-1.735616,-0.771850,0.069953,-0.983518
2,-0.587693,-1.094542,1.324177,1.230212,-0.558611,-1.396535,0.481754,0.275392,0.158672,0.847995,...,-0.115145,0.913093,-0.689110,-1.737536,-1.282519,0.552907,-0.459722,-0.610745,-0.892191,-0.612738
3,-0.236388,-0.575726,1.019683,1.107266,0.259840,-1.400006,0.594474,-0.132596,0.824346,1.230291,...,0.633403,0.724018,-0.899937,-1.351274,-0.283463,0.259046,-1.783027,-0.682162,-0.028057,-0.374174
4,-0.343429,-0.115211,0.708407,0.661426,0.781023,0.275967,0.116875,-0.104860,1.148829,1.382743,...,-0.646476,0.465267,-1.019134,-0.562179,0.867716,0.507593,-0.658179,-0.610955,1.204895,-0.287787
5,0.748761,0.286253,0.327534,0.184658,1.145924,0.505944,0.907674,0.657046,0.830615,1.112297,...,-0.757614,0.502539,-1.416028,0.215477,1.294438,0.359983,0.694093,-0.302558,1.517362,-0.296012
6,-0.082245,0.597984,-0.189503,-0.219884,1.082248,0.693473,-0.236993,1.615453,-0.215960,0.282052,...,0.345772,0.224196,-1.041147,0.949602,1.129754,0.202863,0.289666,-0.320558,1.188206,-0.418208
7,-0.030328,0.806234,-0.656965,-0.723606,0.683738,0.887438,-0.809928,0.558230,-0.940033,-0.632425,...,1.298413,-0.006515,0.539094,0.963541,0.707452,0.625820,0.506229,-0.478101,0.534802,-0.663870
8,1.894997,0.941752,-1.045917,-1.009627,0.458458,0.940006,1.634661,-0.835563,-1.089341,-1.093812,...,1.596124,-0.962486,1.411781,0.521492,0.737541,0.934839,0.128521,-0.551219,-0.132665,1.853862
9,1.292405,0.973717,-1.232918,-1.125922,0.321024,0.840103,0.532886,-1.495939,-1.153056,-1.335633,...,-0.048438,-1.225469,1.087092,0.601303,0.234984,0.146908,0.598893,0.277536,-0.491134,1.569968


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     112.7290
1     151.5420
2     161.9700
3     261.1200
4     250.5190
5     227.2430
6     198.0696
7     166.8504
8     135.0210
9     123.3470
10    120.1530
11    133.5230
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Amapá - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Amapá - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Longevidade
12,-2.04338,0.409525,-0.926633,-1.322522,-1.033191,0.589352,-2.665449,1.108405,2.635823,-0.189959,...,-2.260447,-1.348838,1.830571,0.75466,-2.18432,-2.362189,1.00512,2.208709,-1.813223,-0.150915


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    157.621
Name: Amapá - Consumo de Cimento (t), dtype: float64

In [34]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [35]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [36]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[2816095681, 1603640237, 2568471854, 2763019082, 3324049188, 4054121285, 3221388028, 2701789467, 2139497710, 1315328209, 363789184, 2014461247, 364799340, 2889858145, 3785553595, 451590259, 372399230, 480706128, 380708171, 3284571163, 2976558916, 1098712117, 3970747131, 3869547658, 2063875255, 3674567743, 562605378, 3134381430, 1015038620, 644338688, 2348008216, 969260619, 1213479353, 2616578393, 3034477655, 3079395082, 2610706573, 3299390548, 232858394, 2606076066, 864299609, 45016341, 2649788348, 2133225554, 1502133044, 2404208833, 4067138602, 1885609033, 1711619758, 3370882525, 3677670867, 1256123805, 3277849082, 1812959253, 2373785438, 36643838, 1814283273, 74701428, 1775105913, 2060625008, 1064535645, 2881483469, 915993656, 2232053112, 1392650110, 395545862, 3128596288, 350202419, 2130882579, 4219253898, 354977666, 1814621314, 2511938902, 3905663346, 3366260728, 3359419957, 4008531152, 2860846602, 1678927202, 2898489324, 1915972342, 2367596076, 1543749257, 2221474274, 3088656049, 

val_loss: 60.68470764160156


Step: 83 ___________________________________________
val_loss: 0.29605913162231445


Step: 84 ___________________________________________
val_loss: 22.332748413085938


Step: 85 ___________________________________________
val_loss: 26.916330337524414


Step: 86 ___________________________________________
val_loss: 29.863683700561523


Step: 87 ___________________________________________
val_loss: 0.0019675700459629297


Step: 88 ___________________________________________
val_loss: 53.00204849243164


Step: 89 ___________________________________________
val_loss: 36.21510314941406


Step: 90 ___________________________________________
val_loss: 8.950009942054749e-07


Step: 91 ___________________________________________
val_loss: 0.9360834360122681


Step: 92 ___________________________________________
val_loss: 5.960464477539063e-08


Step: 93 ___________________________________________
val_loss: 0.6516541242599487


Step: 94 _____________________________

In [37]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 472ms/step - loss: 31384.5371 - val_loss: 21290.4941
Epoch 2/10000
1/1 [==============================] - 0s 13ms/step - loss: 29235.6309 - val_loss: 5582.5176
Epoch 3/10000
1/1 [==============================] - 0s 12ms/step - loss: 34717.8125 - val_loss: 20312.3652
Epoch 4/10000
1/1 [==============================] - 0s 13ms/step - loss: 28506.8906 - val_loss: 34854.8047
Epoch 5/10000
1/1 [==============================] - 0s 13ms/step - loss: 25799.2949 - val_loss: 29063.3301
Epoch 6/10000
1/1 [==============================] - 0s 13ms/step - loss: 25229.7324 - val_loss: 34420.8711
Epoch 7/10000
1/1 [==============================] - 0s 13ms/step - loss: 23468.8730 - val_loss: 33744.3477
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 22771.5234 - val_loss: 27425.3027
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 22431.7109 - val_loss: 33389.7266
Epoch 10/10000
1/1 [========

1/1 [==============================] - 0s 13ms/step - loss: 6302.8359 - val_loss: 13983.9482
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 6245.8130 - val_loss: 13782.2100
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 6185.4663 - val_loss: 13667.8955
Epoch 79/10000
1/1 [==============================] - 0s 13ms/step - loss: 6128.1313 - val_loss: 13519.8457
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 6070.8438 - val_loss: 13357.1494
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 6014.6348 - val_loss: 13207.1621
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 5959.1333 - val_loss: 13058.2861
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 5904.3242 - val_loss: 12910.8682
Epoch 84/10000
1/1 [==============================] - 0s 13ms/step - loss: 5850.2046 - val_loss: 12764.7178
Epoch 85/10000
1/1 [=======================

1/1 [==============================] - 0s 13ms/step - loss: 3762.0315 - val_loss: 2412.6240
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 3724.6858 - val_loss: 2364.4441
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 3688.3193 - val_loss: 2338.7678
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 3652.4697 - val_loss: 2278.4365
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 3617.1680 - val_loss: 2249.0708
Epoch 157/10000
1/1 [==============================] - 0s 13ms/step - loss: 3582.8640 - val_loss: 2206.7529
Epoch 158/10000
1/1 [==============================] - 0s 13ms/step - loss: 3549.1943 - val_loss: 2170.0071
Epoch 159/10000
1/1 [==============================] - 0s 13ms/step - loss: 3517.0608 - val_loss: 2142.5623
Epoch 160/10000
1/1 [==============================] - 0s 13ms/step - loss: 3483.3054 - val_loss: 2097.0420
Epoch 161/10000
1/1 [=======================

1/1 [==============================] - 0s 13ms/step - loss: 2164.6865 - val_loss: 637.9856
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 2154.3679 - val_loss: 625.6464
Epoch 230/10000
1/1 [==============================] - 0s 13ms/step - loss: 2143.9873 - val_loss: 615.5689
Epoch 231/10000
1/1 [==============================] - 0s 13ms/step - loss: 2133.8616 - val_loss: 604.7227
Epoch 232/10000
1/1 [==============================] - 0s 13ms/step - loss: 2123.9155 - val_loss: 594.0190
Epoch 233/10000
1/1 [==============================] - 0s 13ms/step - loss: 2114.1768 - val_loss: 583.1740
Epoch 234/10000
1/1 [==============================] - 0s 13ms/step - loss: 2104.5347 - val_loss: 573.0229
Epoch 235/10000
1/1 [==============================] - 0s 13ms/step - loss: 2095.1052 - val_loss: 562.5262
Epoch 236/10000
1/1 [==============================] - 0s 13ms/step - loss: 2085.5908 - val_loss: 554.5663
Epoch 237/10000
1/1 [==============================] 

1/1 [==============================] - 0s 13ms/step - loss: 1707.9834 - val_loss: 155.5215
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 1704.9889 - val_loss: 152.5654
Epoch 306/10000
1/1 [==============================] - 0s 13ms/step - loss: 1702.0513 - val_loss: 149.6004
Epoch 307/10000
1/1 [==============================] - 0s 13ms/step - loss: 1699.0844 - val_loss: 147.3329
Epoch 308/10000
1/1 [==============================] - 0s 13ms/step - loss: 1696.2789 - val_loss: 144.0027
Epoch 309/10000
1/1 [==============================] - 0s 13ms/step - loss: 1693.5111 - val_loss: 141.0497
Epoch 310/10000
1/1 [==============================] - 0s 13ms/step - loss: 1690.7628 - val_loss: 138.3032
Epoch 311/10000
1/1 [==============================] - 0s 13ms/step - loss: 1688.0258 - val_loss: 135.8112
Epoch 312/10000
1/1 [==============================] - 0s 13ms/step - loss: 1685.3867 - val_loss: 132.9683
Epoch 313/10000
1/1 [==============================] 

...

Epoch 1004/10000
1/1 [==============================] - 0s 13ms/step - loss: 1444.7445 - val_loss: 2.3218
Epoch 1005/10000
1/1 [==============================] - 0s 13ms/step - loss: 1444.6426 - val_loss: 2.3489
Epoch 1006/10000
1/1 [==============================] - 0s 13ms/step - loss: 1444.5411 - val_loss: 2.3768
Epoch 1007/10000
1/1 [==============================] - 0s 13ms/step - loss: 1444.4404 - val_loss: 2.4047
Epoch 1008/10000
1/1 [==============================] - 0s 13ms/step - loss: 1444.3403 - val_loss: 2.4197
Epoch 1009/10000
1/1 [==============================] - 0s 13ms/step - loss: 1444.2401 - val_loss: 2.4634
Epoch 1010/10000
1/1 [==============================] - 0s 13ms/step - loss: 1444.1407 - val_loss: 2.4896
Epoch 1011/10000
1/1 [==============================] - 0s 13ms/step - loss: 1444.0420 - val_loss: 2.5192
Epoch 1012/10000
1/1 [==============================] - 0s 13ms/step - loss: 1443.9434 - val_loss: 2.5468
Epoch 1013/10000
1/1 [========================

1/1 [==============================] - 0s 13ms/step - loss: 1438.2106 - val_loss: 4.7361
Epoch 1082/10000
1/1 [==============================] - 0s 13ms/step - loss: 1438.1416 - val_loss: 4.7711
Epoch 1083/10000
1/1 [==============================] - 0s 13ms/step - loss: 1438.0719 - val_loss: 4.8039
Epoch 1084/10000
1/1 [==============================] - 0s 13ms/step - loss: 1438.0031 - val_loss: 4.8385
Epoch 1085/10000
1/1 [==============================] - 0s 13ms/step - loss: 1437.9463 - val_loss: 4.8011
Epoch 1086/10000
1/1 [==============================] - 0s 13ms/step - loss: 1437.8668 - val_loss: 4.9203
Epoch 1087/10000
1/1 [==============================] - 0s 13ms/step - loss: 1437.7988 - val_loss: 4.9458
Epoch 1088/10000
1/1 [==============================] - 0s 13ms/step - loss: 1437.7316 - val_loss: 4.9819
Epoch 1089/10000
1/1 [==============================] - 0s 13ms/step - loss: 1437.6644 - val_loss: 5.0150
Epoch 1090/10000
1/1 [==============================] - 0s 13ms

1/1 [==============================] - 0s 13ms/step - loss: 1433.7521 - val_loss: 7.4887
Epoch 1159/10000
1/1 [==============================] - 0s 13ms/step - loss: 1433.7046 - val_loss: 7.5259
Epoch 1160/10000
1/1 [==============================] - 0s 13ms/step - loss: 1433.6573 - val_loss: 7.5619
Epoch 1161/10000
1/1 [==============================] - 0s 13ms/step - loss: 1433.6108 - val_loss: 7.6007
Epoch 1162/10000
1/1 [==============================] - 0s 13ms/step - loss: 1433.5638 - val_loss: 7.6337
Epoch 1163/10000
1/1 [==============================] - 0s 13ms/step - loss: 1433.5171 - val_loss: 7.6579
Epoch 1164/10000
1/1 [==============================] - 0s 13ms/step - loss: 1433.4711 - val_loss: 7.7096
Epoch 1165/10000
1/1 [==============================] - 0s 13ms/step - loss: 1433.4248 - val_loss: 7.7432
Epoch 1166/10000
1/1 [==============================] - 0s 13ms/step - loss: 1433.3798 - val_loss: 7.7951
Epoch 1167/10000
1/1 [==============================] - 0s 13ms

1/1 [==============================] - 0s 13ms/step - loss: 1430.7081 - val_loss: 10.2928
Epoch 1236/10000
1/1 [==============================] - 0s 13ms/step - loss: 1430.6753 - val_loss: 10.3238
Epoch 1237/10000
1/1 [==============================] - 0s 13ms/step - loss: 1430.6429 - val_loss: 10.3593
Epoch 1238/10000
1/1 [==============================] - 0s 13ms/step - loss: 1430.6113 - val_loss: 10.3599
Epoch 1239/10000
1/1 [==============================] - 0s 13ms/step - loss: 1430.5791 - val_loss: 10.4374
Epoch 1240/10000
1/1 [==============================] - 0s 13ms/step - loss: 1430.5472 - val_loss: 10.4668
Epoch 1241/10000
1/1 [==============================] - 0s 13ms/step - loss: 1430.5156 - val_loss: 10.5032
Epoch 1242/10000
1/1 [==============================] - 0s 13ms/step - loss: 1430.4740 - val_loss: 10.4667
Epoch 1243/10000
1/1 [==============================] - 0s 13ms/step - loss: 1430.4530 - val_loss: 10.5842
Epoch 1244/10000
1/1 [==============================] 

1/1 [==============================] - 0s 13ms/step - loss: 1428.6505 - val_loss: 12.9367
Epoch 1312/10000
1/1 [==============================] - 0s 13ms/step - loss: 1428.6295 - val_loss: 13.0103
Epoch 1313/10000
1/1 [==============================] - 0s 13ms/step - loss: 1428.6061 - val_loss: 12.9956
Epoch 1314/10000
1/1 [==============================] - 0s 13ms/step - loss: 1428.5840 - val_loss: 13.0353
Epoch 1315/10000
1/1 [==============================] - 0s 13ms/step - loss: 1428.5621 - val_loss: 13.0705
Epoch 1316/10000
1/1 [==============================] - 0s 13ms/step - loss: 1428.5404 - val_loss: 13.1014
Epoch 1317/10000
1/1 [==============================] - 0s 13ms/step - loss: 1428.5172 - val_loss: 13.1375
Epoch 1318/10000
1/1 [==============================] - 0s 13ms/step - loss: 1428.4969 - val_loss: 13.1677
Epoch 1319/10000
1/1 [==============================] - 0s 13ms/step - loss: 1428.4756 - val_loss: 13.2025
Epoch 1320/10000
1/1 [==============================] 

In [38]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 35ms/step
[157.621](test_target) - [[133.52304]](prediction) = 24.097959228515634


In [39]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            train_input = train_input.fillna(0.0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [40]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Amapá - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Amapá - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Longevidade
0,-1.262830,-0.924099,-1.323126,-0.093200,-0.714550,-0.289856,-1.309266,1.148205,-1.122043,-1.121418,...,1.412346,1.235380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.282574
1,0.080102,-0.465059,0.229123,-1.175483,-0.699637,1.343672,0.191634,0.140893,-0.184474,-0.185492,...,-0.643250,-0.021555,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,0.125291
2,1.182728,1.389158,1.094002,1.268682,1.414187,-1.053816,1.117631,-1.289098,1.306518,1.306910,...,-0.769096,-1.213825,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0,1.157284


train_target:


,Amapá - Consumo de Cimento (t)
0,112.729
1,151.542
2,161.970


test_input:


...

,Amapá - IDH Educacao,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Amapá - IDH Renda,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amapá - IDH Longevidade
12,-2.04338,0.409525,-0.926633,-1.322522,-1.033191,0.589352,-2.665449,1.108405,2.635823,-0.189959,...,-2.260447,-1.348838,1.830571,0.75466,-2.18432,-2.362189,1.00512,2.208709,-1.813223,-0.150915


test_target:


,Amapá - Consumo de Cimento (t)
12,157.621


1/1 [==============================] - 0s 37ms/step
Error: 24.097959228515634




[161.96998596191406,
 211.83877563476562,
 252.32669067382812,
 244.6859588623047,
 180.22125244140625,
 166.8141632080078,
 135.02467346191406,
 136.92408752441406,
 120.07396697998047,
 133.52304077148438]

In [41]:
display(targets)
display(predictions)

[261.12,
 250.519,
 227.243,
 198.0696,
 166.85039999999998,
 135.021,
 123.347,
 120.153,
 133.523,
 157.621]

[161.96998596191406,
 211.83877563476562,
 252.32669067382812,
 244.6859588623047,
 180.22125244140625,
 166.8141632080078,
 135.02467346191406,
 136.92408752441406,
 120.07396697998047,
 133.52304077148438]

In [42]:
mae = mean_absolute_error(predictions, targets)
mae

32.06900568237305

In [43]:
porcentage = mae/np.mean(targets)
porcentage

0.18082662763035937